In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_path = "mistralai/Mistral-7B-Instruct-v0.2"
base_model = AutoModelForCausalLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
# load data/val/mushroom.en-val.v2.jsonl
import json
import os

data_dir = "data/val"
data_file = "mushroom.en-val.v2.jsonl"
data_path = os.path.join(data_dir, data_file)

with open(data_path, "r") as f:
    data = [json.loads(line) for line in f]

print(data[0])

In [3]:
input = data[0]['model_input'] + " " + data[0]['model_output_text']
input_ids = tokenizer.encode(input, return_tensors="pt")

# print the probability distribution of all tokens
#output = base_model(input_ids, return_dict=True)
#print(output.logits.shape)

In [ ]:
sum = 0
for i, token in enumerate(tokenizer.convert_ids_to_tokens(input_ids[0])):
    if i == 0 or i == len(input_ids[0]) - 1:
        continue
    # get length of token
    print(token)
    length = len(token)
    sum += length

In [ ]:
len(input), sum

In [ ]:
input_ids[0]

In [ ]:
# map token to the relative bound of the token in the input
boundaries = data[0]['hard_labels']
boundaries = [(start+len(data[0]['model_input']), end+len(data[0]['model_input'])) for start, end in boundaries]
print(boundaries)

tokens = []
# for each boundary, find the corresponding token
for boundary in boundaries:
    start, end = boundary
    token = input[start:end+1]
    print(token)
    tokens.append(token)

    # append token idx 

print(len(tokenizer.convert_ids_to_tokens(input_ids[0])), tokenizer.convert_ids_to_tokens(input_ids[0]))

In [ ]:
idx_to_evaluate = 20  # 22, 26, 34

# get the probability distribution of the token
tokenizer.convert_ids_to_tokens(int(input_ids[0][idx_to_evaluate])), tokenizer.convert_ids_to_tokens(int(input_ids[0][idx_to_evaluate+1]))

In [ ]:
sentence_1 = tokenizer.convert_ids_to_tokens(int(input_ids[0][idx_to_evaluate]))
sentence_1

In [ ]:
get_mnli_label(sentence_1, sentence_2, model, tokenizer)

In [ ]:
topk = 10 

# get the topk tokens
print(output.logits[0][idx_to_evaluate].topk(topk).indices.shape)
topk_tokens = tokenizer.convert_ids_to_tokens(output.logits[0][idx_to_evaluate].topk(topk).indices)
print(topk_tokens)
# print predicted token
print(tokenizer.convert_ids_to_tokens(int(input_ids[0][idx_to_evaluate+1])))

In [ ]:
output_logits = output.logits[0][idx_to_evaluate]
# softmax the logits
output_logits = output_logits.softmax(dim=0)
# get the probability of the predicted token
# summ all the probabilities of the topk tokens
sum = 0
for i in range(10):
    token = topk_tokens[i]
    idx = tokenizer.convert_tokens_to_ids(token)
    prob = output_logits[idx].item()
    print(token, prob)

    
    sum += prob
print(sum)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("cross-encoder/nli-deberta-v3-xsmall")
model = AutoModelForSequenceClassification.from_pretrained("cross-encoder/nli-deberta-v3-xsmall")

In [ ]:
# make an nli prediction
sentence_1 = "gold"
sentence_2 = "silver"

# encode the sentences
def get_mnli_label(sentence_1, sentence_2, model, tokenizer):
    inputs = tokenizer(sentence_1, sentence_2, return_tensors="pt")
    # make a prediction
    outputs = model(**inputs)
    # get the predicted class
    predicted_class_idx = outputs.logits.argmax().item()
    # get the predicted class name
    predicted_class_name = model.config.id2label[predicted_class_idx]
    return predicted_class_name

get_mnli_label(sentence_1, sentence_2, model, tokenizer)

In [ ]:
print(input)
from termcolor import colored
import itertools
print(" " * len(data[0]['model_input']))

def print_colored_text(text):
    colors = ['red', 'green', 'yellow', 'blue', 'magenta', 'cyan', 'white']
    color_cycle = itertools.cycle(colors)
    
    for i in range(0, len(text), 10):
        color = next(color_cycle)
        print(colored(text[i:i+10], color), end='')
    print()

# Esempio di utilizzo
text = " " * (len(data[0]['model_input'])+1) + "1234567890" * 9
print_colored_text(text)

In [ ]:
output.logits[0][i]

In [ ]:
for i in range(output.logits.shape[1]):
    print(i)
    print(input_ids[0][i], tokenizer.decode(input_ids[0][i].item()), output.logits[0, i].argmax().item(), tokenizer.decode(output.logits[0][i].argmax().item()))

In [ ]:
index = 27
output.logits[0][index]

# get the top k tokens 
import torch
import torch.nn.functional as F

index = 27
top_k = 5
values, indices = torch.topk(F.softmax(output.logits[0][index], dim=0), top_k)
print(values, indices)


In [ ]:
# print the top k tokens
for i in range(top_k):
    print(tokenizer.decode(indices[i].item()), values[i].item())